In [1]:
# FOR SPLITTING DATASET
# Import os to move, modify and create directory for training set and test set
import os
# Import random to generate randomicaly a number
import random
# Import shutil to move the image 
import shutil

# FOR DATA LOADER 
from tensorflow import keras
from keras.utils import image_dataset_from_directory
from keras.src.legacy.preprocessing.image import ImageDataGenerator
import numpy as np

# FOR AUXILIARY FUNCTION OF NEURAL NETWORK

from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.models import load_model
from sklearn.metrics import classification_report
import matplotlib.pyplot as plt
import numpy as np
import cv2

# FOR MODELS NEURAL NETWORK
from keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPooling2D, Flatten
from tensorflow.keras import backend as k

# FOR TRAINING 
from keras.utils import plot_model
import visualkeras


In [2]:
from tensorflow.config.experimental import list_physical_devices, set_memory_growth
# Avoid OOM errors by setting GPU Memory Consumption Growth
gpus =list_physical_devices('GPU')
for gpu in gpus: 
    set_memory_growth(gpu, True)

In [4]:
import os
import random
import shutil

def split_dataset(source_dir, train_dir, val_dir, test_ratio=0.2):
    """
    Suddivide un dataset in training set e validation set.
    
    Args:
        source_dir (str): Path alla directory sorgente contenente le sottocartelle di classi.
        train_dir (str): Path della directory per il training set.
        val_dir (str): Path della directory per il validation set.
        split_ratio (float): Proporzione di immagini da includere nel training set (default: 0.8).
    
    Returns:
        None
    """
    # Crea le directory per training e validation set, se non esistono
    os.makedirs(train_dir, exist_ok=True)
    os.makedirs(val_dir, exist_ok=True)

    # Itera su tutte le sottocartelle nella directory sorgente
    class_dirs = [d for d in os.listdir(source_dir) if os.path.isdir(os.path.join(source_dir, d))]
    
    for class_name in class_dirs:
        source_class_path = os.path.join(source_dir, class_name)
        
        # Percorsi delle sottodirectory per la classe
        train_class_path = os.path.join(train_dir, class_name)
        val_class_path = os.path.join(val_dir, class_name)
        os.makedirs(train_class_path, exist_ok=True)
        os.makedirs(val_class_path, exist_ok=True)

        # Elenco di tutti i file immagine nella sottodirectory della classe
        all_images = [f for f in os.listdir(source_class_path) if os.path.isfile(os.path.join(source_class_path, f))]
        
        # Mescola casualmente le immagini
        random.shuffle(all_images)

        # Determina il numero di immagini per il training set
        split_index = int(len(all_images) * test_ratio)
        train_images = all_images[:split_index]
        val_images = all_images[split_index:]

        # Sposta i file nel training set
        for image in train_images:
            shutil.move(
                os.path.join(source_class_path, image),
                os.path.join(train_class_path, image)
            )

        # Sposta i file nel validation set
        for image in val_images:
            shutil.move(
                os.path.join(source_class_path, image),
                os.path.join(val_class_path, image)
            )

        print(f"Classe '{class_name}': {len(train_images)} immagini nel training set, {len(val_images)} nel validation set.")

    print("Suddivisione del dataset completata.")




In [5]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

def load_dataset(training_path, validation_path, target_size, batch_size):
    """
    Carica i dati di training e validation applicando data augmentation.
    """
    # Creazione del generatore di immagini con data augmentation
    augmentation = ImageDataGenerator(
        rescale=1./255,        # Normalizzazione dei valori dei pixel
        shear_range=0.2,       # Distorsione delle immagini
        zoom_range=0.2,        # Applicazione di zoom casuale
        horizontal_flip=True   # Inversione orizzontale casuale
    )

    print('\n--- Preparazione del TRAINING SET ---')
    train_data = augmentation.flow_from_directory(
        training_path,
        target_size=(target_size[0], target_size[1]),  # Ridimensiona immagini
        batch_size=batch_size,                         # Batch size per training
        class_mode='categorical',                      # Classificazione multiclasse
        color_mode='rgb'                               # Immagini a colori
    )

    print('\n--- Preparazione del VALIDATION SET ---')
    val_data = augmentation.flow_from_directory(
        validation_path,
        target_size=(target_size[0], target_size[1]),  # Ridimensiona immagini
        batch_size=batch_size,                         # Batch size per validation
        class_mode='categorical',                      # Classificazione multiclasse
        color_mode='rgb'                               # Immagini a colori
    )
    
    return train_data, val_data




In [6]:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

def train_fungi_model(model, train_data, validation_data, use_early_stopping, patience, model_id, epochs):
    """
    Addestra il modello utilizzando i set di training e validation.
    Supporta l'early stopping per evitare overfitting.
    """
    # Configurazione Early Stopping
    early_stop = None
    if use_early_stopping:
        early_stop = EarlyStopping(
            monitor='val_loss',            # Monitora la perdita sulla validation
            mode='min',                    # Minimizziamo la perdita
            patience=patience,             # Numero di epoche senza miglioramenti
            restore_best_weights=True,     # Ripristina i pesi migliori
            verbose=1                      # Mostra messaggi durante il training
        )

    # Configurazione del checkpoint per salvare il modello
    model_filename = f"fungi_model_{model_id}.h5"
    checkpoint = ModelCheckpoint(
        filepath=model_filename,           # Salva il file del modello
        monitor='loss',                    # Monitora la perdita sul training
        verbose=1,                         # Mostra messaggi durante il salvataggio
        save_best_only=True,               # Salva solo il miglior modello
        mode='min'                         # Minimizziamo la perdita
    )

    # Lista delle callback
    callbacks = [checkpoint]
    if early_stop:
        callbacks.append(early_stop)

    # Avvio del training del modello
    history = model.fit(
        train_data,                        # Dati di training
        validation_data=validation_data,  # Dati di validation
        callbacks=callbacks,               # Callback configurati
        epochs=epochs                      # Numero di epoche
    )

    return history, model_filename, early_stop






def evaluate_fungi_model(model, training_data, validation_data):
    """
    Valuta il modello sui set di training e validation.
    Stampa l'accuratezza e la perdita per entrambi i set.
    """
    print("--- Valutazione del modello ---")
    
    # Valutazione sul training set
    train_metrics = model.evaluate(training_data, verbose=1)
    train_loss, train_accuracy = train_metrics[0], train_metrics[1]
    print(f"Training Set - Accuratezza: {train_accuracy:.4f}, Perdita: {train_loss:.4f}")
    
    # Valutazione sul validation set
    val_metrics = model.evaluate(validation_data, verbose=1)
    val_loss, val_accuracy = val_metrics[0], val_metrics[1]
    print(f"Validation Set - Accuratezza: {val_accuracy:.4f}, Perdita: {val_loss:.4f}")
    
    return {
        "training": {"accuracy": train_accuracy, "loss": train_loss},
        "validation": {"accuracy": val_accuracy, "loss": val_loss}
    }


In [8]:
import matplotlib.pyplot as plt

def plot_loss(history):
    """
    Traccia la curva della perdita (loss) per il training e la validation.
    """
    plt.figure(figsize=(10, 6))
    plt.plot(history.history['loss'], label='Training Loss', color='blue', linewidth=2)
    plt.plot(history.history['val_loss'], label='Validation Loss', color='red', linewidth=2)
    plt.title('Andamento della Perdita', fontsize=16)
    plt.xlabel('Epoche', fontsize=14)
    plt.ylabel('Perdita', fontsize=14)
    plt.legend(loc='upper right', fontsize=12)
    plt.grid(True, linestyle='--', alpha=0.6)
    plt.show()

def plot_accuracy(history):
    """
    Traccia la curva dell'accuratezza (accuracy) per il training e la validation.
    """
    plt.figure(figsize=(10, 6))
    plt.plot(history.history.get('accuracy', history.history.get('acc')), label='Training Accuracy', color='green', linewidth=2)
    plt.plot(history.history.get('val_accuracy', history.history.get('val_acc')), label='Validation Accuracy', color='orange', linewidth=2)
    plt.title('Andamento dell\'Accuratezza', fontsize=16)
    plt.xlabel('Epoche', fontsize=14)
    plt.ylabel('Accuratezza', fontsize=14)
    plt.legend(loc='lower right', fontsize=12)
    plt.grid(True, linestyle='--', alpha=0.6)
    plt.show()

def plot_training_history(history):
    """
    Traccia sia la perdita che l'accuratezza in due grafici separati.
    """
    print("Generazione dei grafici di perdita e accuratezza...")
    plot_loss(history)
    plot_accuracy(history)


In [9]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.models import load_model

def test_model(model, train_set, input_size, test_image_path):
    """
    Testa un'immagine singola utilizzando il modello addestrato.
    
    Args:
        model: Il modello addestrato o il percorso al file del modello salvato.
        train_set: Il set di training per recuperare le classi.
        input_size: Dimensione dell'input del modello (es. (224, 224)).
        test_image_path: Percorso dell'immagine da testare.

    Returns:
        Classe predetta come stringa.
    """
    # Mappa numeri alle classi (es. indice numerico → nome della specie)
    class_mapping = {v: k for k, v in train_set.class_indices.items()}
    
    # Legge e preprocessa l'immagine di test
    test_image = cv2.imread(test_image_path)
    if test_image is None:
        raise FileNotFoundError(f"Immagine non trovata: {test_image_path}")

    test_image = cv2.resize(test_image, input_size)  # Ridimensiona l'immagine
    test_image = cv2.cvtColor(test_image, cv2.COLOR_BGR2RGB)  # Converti da BGR a RGB
    plt.imshow(test_image)  # Mostra l'immagine originale
    plt.axis('off')
    plt.show()

    # Prepara l'immagine per il modello
    test_image = np.expand_dims(test_image, axis=0)  # Aggiunge una dimensione per il batch
    test_image = test_image / 255.0  # Normalizza i valori dei pixel

    # Carica il modello se necessario
    if isinstance(model, str):
        print(f"Caricamento modello da: {model}")
        model = load_model(model)

    # Predice la classe dell'immagine
    prediction_prob = model.predict(test_image)
    predicted_class_index = np.argmax(prediction_prob, axis=1)[0]
    predicted_class = class_mapping[predicted_class_index]

    # Mostra il risultato
    plt.title(f"Predizione: {predicted_class}")
    plt.show()

    return predicted_class
